### Whole file process

### Construct business dictionary

In [98]:
import pandas as pd
import json
def construct_business_dict():
    PATH = '/Users/fangzheng/Downloads/archive 2/yelp_academic_dataset_business.json'
    data = pd.read_json(PATH, lines=True)
    data = data[['business_id']]
    data
    return data

In [99]:
business_id_df = construct_business_dict()

In [100]:
print(type(business_id_df['business_id']))
business_id_df['business_id'] = business_id_df['business_id'].astype(str)

<class 'pandas.core.series.Series'>


### Construct User dictionary

In [101]:

def construct_user_dict():
    PATH = '/Users/fangzheng/Downloads/archive 2/yelp_academic_dataset_user.json'

    # 用 Pandas 读取 user.json 文件
    # with open(PATH) as f:
        # df = json.load(f)
    data_list = []
    with open(PATH,'r') as f:
        for line in f:
            data = json.loads(line)  # 解析每一行的JSON数据
            user_id = data['user_id']  # 获取 user_id
            name = data['name']
            if user_id and name:
                data_list.append({'user_id': user_id, 'name': name})
    df = pd.DataFrame(data_list)
    user_data = df.groupby('name')['user_id'].apply(list).to_dict()
    user_data = dict(sorted(user_data.items()))

    return user_data


In [102]:
from fuzzywuzzy import fuzz
similarity_threshold = 50  # 
user_data = construct_user_dict()

In [103]:
user_data
user_data = dict(sorted(user_data.items()))


In [104]:
import json
import pandas as pd
import random
import re

pattern = r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

def data_preprocessing(PATH):
    # load data from 1.json
    with open(PATH) as f:
        data = json.load(f)
    
    # convert data to pandas dataframe
    review_df = pd.DataFrame(data)
    # free data
    del data
    review_df = review_df.rename(columns={
        'Date': 'date',
        'Place': 'place',
        'User': 'name',
        'Rating': 'stars',
        'Review': 'text',

    })

    review_df['stars'] = review_df['stars'].apply(split)
    #  'date' 
    review_df['date'] = pd.to_datetime(review_df['date'], format='%b %d, %Y')
    # to 'YYYY-MM-DD' format
    review_df['date'] = review_df['date'].dt.strftime('%Y-%m-%d')
    review_df['date'] = review_df['date'].apply(random_time)
    review_df['date'] = pd.to_datetime(review_df['date'],format='%Y-%m-%d%H:%M:%S')


    # 
    review_df['name'] = review_df['name'].apply(lambda x: re.sub(pattern, '', x))

    review_df['name'] = review_df['name'].apply(lambda x: re.sub(r'\s+', ' ', x))
    
    review_df['user_id'] = review_df['name'].apply(find_matching_user)

    review_df['user_id'] = review_df['user_id'].apply(random_select)
    review_df['user_id'] = review_df['user_id'].astype(str)
    review_df = review_df[['user_id','name', 'stars', 'useful','funny','cool','text', 'place', 'date']]
    review_df['useful'] = review_df['useful'].astype(int)
    review_df['funny'] = review_df['funny'].astype(int)
    review_df['cool'] = review_df['cool'].astype(int)

    return review_df

def random_time(row):
    random_hours = random.randint(0, 23)
    random_minutes = random.randint(0, 59)
    random_seconds = random.randint(0, 59)
    random_time_str = f"{random_hours:02d}:{random_minutes:02d}:{random_seconds:02d}"
    return str(row + random_time_str)

def split(text):
    words = text.split(' ')
    return float(words[0])


def find_matching_user(name):
    matched_user_id = None
    for user_name, user_ids in user_data.items():
        similarity = fuzz.ratio(name, user_name)
        if similarity > similarity_threshold:
            matched_user_id = user_ids
            break
    return matched_user_id

def random_select(row):
    return random.choice(row)



In [105]:
def save_as_review_json(new_df,word):
    output_path = '/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/my_reviews.json'
    try:
        existing_df = pd.read_json(output_path, lines=True)
    except FileNotFoundError:
        existing_df = pd.DataFrame()


    merged_df = pd.concat([existing_df, new_df], ignore_index=True)

    merged_df.to_json(output_path, orient='records', lines=True)

    print(f"Appended and saved JSON file at: {output_path}, Orginal json is :{word}.json")

* load all files in reviews folder

In [106]:
# 
import os
base_path = os.getcwd()
base_path
files = os.listdir(base_path)

for i in range(len(files)):
    if files[i].endswith('.json'):
        path = os.path.join(base_path, files[i])
        print(path)
        index = int(files[i].split('.')[0])  # 就是文件名+1就是之前business.json的行数, 因为counter是从零开始的
        # print(index)
        print(f'this is {i},index: {index}')

        review_df = data_preprocessing(path)
        review_df['business_id'] = business_id_df['business_id'][index]
        review_df = review_df[['business_id','user_id','name', 'stars', 'useful','funny','cool','text', 'place', 'date']]
        # print(review_df['date'])
        save_as_review_json(review_df,index)
        

/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/reviews/729.json
this is 0,index: 729
Appended and saved JSON file at: /Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/my_reviews.json, Orginal json is :729.json
/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/reviews/16030.json
this is 1,index: 16030
Appended and saved JSON file at: /Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/my_reviews.json, Orginal json is :16030.json
/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/reviews/83364.json
this is 2,index: 83364
Appended and saved JSON file at: /Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/my_reviews.json, Orginal json is :83364.json
/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/reviews/6523.json
this is 3,index: 6523
Appended and saved JSON file at: /Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/my_reviews.json, Orginal json is :6523.json
/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/reviews/30102.js

KeyboardInterrupt: 

In [3]:
import pandas as pd
data = pd.read_json('/Users/fangzheng/Documents/DSML_sem1/DSA5103/assignment3/my_reviews.json',lines=True)

,business_id,user_id,name,stars,useful,funny,cool,text,place,date
0,ZY4R4YD3uGmzyECzEdr1rg,qk3En2UYebrQCYtNf1ml1g,Brett M.,5,0,0,0,I took the PMP Bootcamp with Stephen Hoogerwer...,Location,2022-03-03 01:36:46
1,ZY4R4YD3uGmzyECzEdr1rg,_to0QmN-ySR33K_KA29ZRA,Jane X.,5,1,0,1,I attended the PMP Exam Prep Bootcamp in India...,"Indianapolis, IN",2020-02-04 00:27:41
2,ZY4R4YD3uGmzyECzEdr1rg,SHLkRNerzO9MwzEZKzcoxw,Michael M.,5,0,0,0,I attended PMA's Risk Management Professional ...,"Fishers, IN",2018-01-31 22:52:10
3,ZY4R4YD3uGmzyECzEdr1rg,7tvuYlCaJY1q05DVqdaHwQ,Ashleigh A.,5,0,0,0,"New to project management, I took PMA's PMP Bo...","San Antonio, TX",2017-05-10 13:52:23
4,ZY4R4YD3uGmzyECzEdr1rg,Z4pdbx8k4wJRorAsp8SeaA,Miguel R.,5,0,0,0,I attended the PMP Bootcamp and a week later t...,"Indianapolis, IN",2016-06-29 22:38:12
...,...,...,...,...,...,...,...,...,...,...
49685,K0nNsUGPoE7ezIXiBwxrFQ,f0YhTAOTzTaV4G4XB7OgFw,Rodger T.,1,0,0,0,Arhaus does not stand behind the quality of th...,"Indianapolis, IN",2018-12-26 16:30:56
49686,K0nNsUGPoE7ezIXiBwxrFQ,H5jODGYObVpKfXD_yetEeA,Michelle M.,1,0,0,0,Terrible experience so far!! Ordered 2 swivel...,"Columbus, IN",2021-04-14 06:15:03
49687,K0nNsUGPoE7ezIXiBwxrFQ,tcrwCMQKrV0bSVDF0b6ELQ,Lindsey C.,5,4,0,2,If you're looking for industrial chic furnitur...,"Mile Square, Indianapolis, IN",2017-06-25 07:53:45
49688,K0nNsUGPoE7ezIXiBwxrFQ,1QFsttUnXeBH0YP3p8oGEg,Bob H.,2,0,0,0,Over the years I have spent many thousands of ...,"Martinsville, IN",2021-02-19 08:49:41


In [4]:
data.columns

Index(['business_id', 'user_id', 'name', 'stars', 'useful', 'funny', 'cool',
       'text', 'place', 'date'],
      dtype='object')

### One File Process

In [2]:
import os
base_path = os.getcwd()
base_path
files = os.listdir(base_path)
print(files)
print(type(files))
# print(f'The number of files: ',len(files))
print(f'The number of files:\t{len(files)}')


['729.json', '16030.json', '83364.json', '6523.json', '30102.json', '86536.json', '6489.json', '3371.json', '683.json', '13662.json', '379.json', '9286.json', '13398.json', '19795.json', '6173.json', '2999.json', '2130.json', '86865.json', '12423.json', '80548.json', '2560.json', '87327.json', '12073.json', '16899.json', '20102.json', '17271.json', '12589.json', '18184.json', '6870.json', '15759.json', '16876.json', '2075.json', '86920.json', '12566.json', '20047.json', '15309.json', '17334.json', '4908.json', '87262.json', '2425.json', '43093.json', '12136.json', '16175.json', '9793.json', '8881.json', '19280.json', '3664.json', '18868.json', '2976.json', '9269.json', '13377.json', '396.json', '14518.json', '32580.json', '6036.json', '83671.json', '1209.json', '13727.json', '9639.json', '12835.json', '80374.json', '816.json', '10222.json', '19900.json', '1823.json', '18612.json', '5163.json', '15470.json', '9843.json', '11960.json', '8751.json', '5499.json', '18242.json', '4322.json',

In [79]:
import json
import pandas as pd
import random
# load data from 1.json
with open('16030.json') as f:
    data = json.load(f)
# convert data to pandas dataframe
review_df = pd.DataFrame(data)
review_df = review_df.rename(columns={
    'Date': 'date',
    'Place': 'place',
    'User': 'name',
    'Rating': 'stars',
    'Review': 'text',

})

def split(text):
    words = text.split(' ')
    return float(words[0])
review_df['stars'] = review_df['stars'].apply(split)
review_df['date'] = pd.to_datetime(review_df['date'], format='%b %d, %Y')
review_df['date'] = review_df['date'].dt.strftime('%Y-%m-%d')
print(review_df)


          date                          place         name  stars useful  \
0   2019-05-25                       Location       Amy G.    4.0      2   
1   2023-09-02             St. Petersburg, FL    Darrin O.    1.0      0   
2   2023-05-04             Tarpon Springs, FL      Brad W.    1.0      0   
3   2023-06-01              Safety Harbor, FL      John S.    1.0      1   
4   2020-12-30                    Orlando, FL     Mirko P.    4.0      3   
5   2020-11-13                  Nashville, TN  Courtney K.    1.0      0   
6   2022-02-13  Temple Terrace, Tampa Bay, FL   Merilee H.    5.0      0   
7   2020-10-28              St Petersburg, FL     Dylan V.    1.0      1   
8   2020-12-11                 Fort Myers, FL         D 1.    1.0      0   
9   2020-05-18             St. Petersburg, FL     Laura J.    1.0      2   
10  2019-05-25                       Location       Amy G.    4.0      2   
11  2023-09-02             St. Petersburg, FL    Darrin O.    1.0      0   
12  2023-05-

* random generate hour minute second to the screwed data

In [80]:
def random_time(row):
    random_hours = random.randint(0, 23)
    random_minutes = random.randint(0, 59)
    random_seconds = random.randint(0, 59)
    random_time_str = f"{random_hours:02d}:{random_minutes:02d}:{random_seconds:02d}"
    return str(row + random_time_str)

In [81]:
review_df.columns
# change the order of columns change date to the last column
review_df = review_df[['name', 'stars', 'useful','funny','cool','text', 'place', 'date']]


In [82]:
review_df
review_df['date'] = review_df['date'].apply(random_time)


In [83]:
review_df['date'] = pd.to_datetime(review_df['date'],format='%Y-%m-%d%H:%M:%S')
review_df

,name,stars,useful,funny,cool,text,place,date
0,Amy G.,4.0,2,0,1,"Great public course, i came here on Saturday f...",Location,2019-05-25 23:23:41
1,Darrin O.,1.0,0,0,0,It's a fairly nice course but not even able to...,"St. Petersburg, FL",2023-09-02 20:33:58
2,Brad W.,1.0,0,0,0,Course is great.. but the clubhouse people suc...,"Tarpon Springs, FL",2023-05-04 13:00:26
3,John S.,1.0,1,0,0,A dump...if your driving range has no grass an...,"Safety Harbor, FL",2023-06-01 07:57:26
4,Mirko P.,4.0,3,0,4,It was a nice round of the golf. The last one ...,"Orlando, FL",2020-12-30 06:10:15
5,Courtney K.,1.0,0,0,0,I have come here multiple times at least 30 mi...,"Nashville, TN",2020-11-13 07:45:53
6,Merilee H.,5.0,0,0,0,This is Jim Hampton's favorite . The course is...,"Temple Terrace, Tampa Bay, FL",2022-02-13 19:03:41
7,Dylan V.,1.0,1,2,0,You are better off hitting golf balls out of y...,"St Petersburg, FL",2020-10-28 08:25:08
8,D 1.,1.0,0,0,0,Rude staff at the clubhouse. If you want to pl...,"Fort Myers, FL",2020-12-11 01:59:58
9,Laura J.,1.0,2,1,0,I don't know what it is about the staff who wo...,"St. Petersburg, FL",2020-05-18 11:57:24


In [84]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   name    40 non-null     object        
 1   stars   40 non-null     float64       
 2   useful  40 non-null     object        
 3   funny   40 non-null     object        
 4   cool    40 non-null     object        
 5   text    40 non-null     object        
 6   place   40 non-null     object        
 7   date    40 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 2.6+ KB


In [85]:
review_df = review_df[['name', 'stars', 'useful','funny','cool','text', 'place', 'date']]
review_df

,name,stars,useful,funny,cool,text,place,date
0,Amy G.,4.0,2,0,1,"Great public course, i came here on Saturday f...",Location,2019-05-25 23:23:41
1,Darrin O.,1.0,0,0,0,It's a fairly nice course but not even able to...,"St. Petersburg, FL",2023-09-02 20:33:58
2,Brad W.,1.0,0,0,0,Course is great.. but the clubhouse people suc...,"Tarpon Springs, FL",2023-05-04 13:00:26
3,John S.,1.0,1,0,0,A dump...if your driving range has no grass an...,"Safety Harbor, FL",2023-06-01 07:57:26
4,Mirko P.,4.0,3,0,4,It was a nice round of the golf. The last one ...,"Orlando, FL",2020-12-30 06:10:15
5,Courtney K.,1.0,0,0,0,I have come here multiple times at least 30 mi...,"Nashville, TN",2020-11-13 07:45:53
6,Merilee H.,5.0,0,0,0,This is Jim Hampton's favorite . The course is...,"Temple Terrace, Tampa Bay, FL",2022-02-13 19:03:41
7,Dylan V.,1.0,1,2,0,You are better off hitting golf balls out of y...,"St Petersburg, FL",2020-10-28 08:25:08
8,D 1.,1.0,0,0,0,Rude staff at the clubhouse. If you want to pl...,"Fort Myers, FL",2020-12-11 01:59:58
9,Laura J.,1.0,2,1,0,I don't know what it is about the staff who wo...,"St. Petersburg, FL",2020-05-18 11:57:24


In [86]:
PATH = '/Users/fangzheng/Downloads/archive 2/yelp_academic_dataset_user.json'

In [33]:
import pandas as pd
import json
data_list = []
with open(PATH,'r') as f:
    for line in f:
        data = json.loads(line)  # 解析每一行的JSON数据
        user_id = data['user_id']  # 获取 user_id
        name = data['name']
        if user_id and name:
            data_list.append({'user_id': user_id, 'name': name})
user_df = pd.DataFrame(data_list)
user_df
# {"user_id":"hA5lMy-EnncsH4JoR-hFGQ","name":"Karen","review_count":79,"yelping_since":"2007-01-05 19:40:59","useful":29,"funny":15,"cool":7,"elite":"","friends

,user_id,name
0,qVc8ODYU5SZjKXVBgXdI7w,Walker
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel
2,2WnXYQFK0hXEoTxPtV2zvg,Steph
3,SZDeASXq7o05mMNLshsdIA,Gwen
4,hA5lMy-EnncsH4JoR-hFGQ,Karen
...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,Jerrold
1987893,68czcr4BxJyMQ9cJBm6C7Q,Jane
1987894,1x3KMskYxOuJCjRz70xOqQ,Shomari
1987895,ulfGl4tdbrH05xKzh5lnog,Susanne


In [36]:
user_data = user_df.groupby('name')['user_id'].apply(list).to_dict()

print(type(user_data))

<class 'dict'>


In [87]:
len(user_data)

132361

In [40]:
user_data = dict(sorted(user_data.items()))

In [88]:
import re
pattern = r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

review_df['name'] = review_df['name'].apply(lambda x: re.sub(pattern, '', x))

review_df['name'] = review_df['name'].apply(lambda x: re.sub(r'\s+', ' ', x))

print(review_df)

           name  stars useful funny cool  \
0        Amy G.    4.0      2     0    1   
1     Darrin O.    1.0      0     0    0   
2       Brad W.    1.0      0     0    0   
3       John S.    1.0      1     0    0   
4      Mirko P.    4.0      3     0    4   
5   Courtney K.    1.0      0     0    0   
6    Merilee H.    5.0      0     0    0   
7      Dylan V.    1.0      1     2    0   
8          D 1.    1.0      0     0    0   
9      Laura J.    1.0      2     1    0   
10       Amy G.    4.0      2     0    1   
11    Darrin O.    1.0      0     0    0   
12      Brad W.    1.0      0     0    0   
13      John S.    1.0      1     0    0   
14     Mirko P.    4.0      3     0    4   
15  Courtney K.    1.0      0     0    0   
16   Merilee H.    5.0      0     0    0   
17     Dylan V.    1.0      1     2    0   
18         D 1.    1.0      0     0    0   
19     Laura J.    1.0      2     1    0   
20       Amy G.    4.0      2     0    1   
21    Darrin O.    1.0      0   

In [92]:
from fuzzywuzzy import fuzz
similarity_threshold = 50  # 
def find_matching_user(name):
    matched_user_id = None
    for user_name, user_ids in user_data.items():
        similarity = fuzz.ratio(name, user_name)
        if similarity > similarity_threshold:
            matched_user_id = user_ids
            break
    return matched_user_id

In [93]:
review_df['user_id'] = review_df['name'].apply(find_matching_user)
review_df

,name,stars,useful,funny,cool,text,place,date,user_id
0,Amy G.,4.0,2,0,1,"Great public course, i came here on Saturday f...",Location,2019-05-25 23:23:41,[I1OuDgYZOknPxp8MURUT1w]
1,Darrin O.,1.0,0,0,0,It's a fairly nice course but not even able to...,"St. Petersburg, FL",2023-09-02 20:33:58,[0D5WmoI0xv-t8PTDwhluhg]
2,Brad W.,1.0,0,0,0,Course is great.. but the clubhouse people suc...,"Tarpon Springs, FL",2023-05-04 13:00:26,[t1MblByl5GkJXIAIPzewQA]
3,John S.,1.0,1,0,0,A dump...if your driving range has no grass an...,"Safety Harbor, FL",2023-06-01 07:57:26,"[yuWOLUWnGu2XFiL_TdNB9w, ul6pwQqpzj-pbmzMghDl7g]"
4,Mirko P.,4.0,3,0,4,It was a nice round of the golf. The last one ...,"Orlando, FL",2020-12-30 06:10:15,[59dpWjJdntwzfIsKOcbQqA]
5,Courtney K.,1.0,0,0,0,I have come here multiple times at least 30 mi...,"Nashville, TN",2020-11-13 07:45:53,[xQBiULOOxAqn_y39Ynckbw]
6,Merilee H.,5.0,0,0,0,This is Jim Hampton's favorite . The course is...,"Temple Terrace, Tampa Bay, FL",2022-02-13 19:03:41,"[MO36VisoAfrUdHrR6UG-_g, QqMQchSLRdodtUlrICXTjw]"
7,Dylan V.,1.0,1,2,0,You are better off hitting golf balls out of y...,"St Petersburg, FL",2020-10-28 08:25:08,[mijVMekO1vG7MbAL78uRNg]
8,D 1.,1.0,0,0,0,Rude staff at the clubhouse. If you want to pl...,"Fort Myers, FL",2020-12-11 01:59:58,"[TcXWl16j3qq0mm5dtCSBpQ, 8DVtKxlLJIayoALUKOthL..."
9,Laura J.,1.0,2,1,0,I don't know what it is about the staff who wo...,"St. Petersburg, FL",2020-05-18 11:57:24,[uYSU2D0byI__7E1nLFx49w]


In [94]:
def random_select(row):
    return random.choice(row)

review_df['user_id'] = review_df['user_id'].apply(random_select)
review_df['user_id']


0     I1OuDgYZOknPxp8MURUT1w
1     0D5WmoI0xv-t8PTDwhluhg
2     t1MblByl5GkJXIAIPzewQA
3     yuWOLUWnGu2XFiL_TdNB9w
4     59dpWjJdntwzfIsKOcbQqA
5     xQBiULOOxAqn_y39Ynckbw
6     QqMQchSLRdodtUlrICXTjw
7     mijVMekO1vG7MbAL78uRNg
8     Guhi3rYZBLaXCwSVNKwSFw
9     uYSU2D0byI__7E1nLFx49w
10    I1OuDgYZOknPxp8MURUT1w
11    0D5WmoI0xv-t8PTDwhluhg
12    t1MblByl5GkJXIAIPzewQA
13    ul6pwQqpzj-pbmzMghDl7g
14    59dpWjJdntwzfIsKOcbQqA
15    xQBiULOOxAqn_y39Ynckbw
16    QqMQchSLRdodtUlrICXTjw
17    mijVMekO1vG7MbAL78uRNg
18    Guhi3rYZBLaXCwSVNKwSFw
19    uYSU2D0byI__7E1nLFx49w
20    I1OuDgYZOknPxp8MURUT1w
21    0D5WmoI0xv-t8PTDwhluhg
22    t1MblByl5GkJXIAIPzewQA
23    yuWOLUWnGu2XFiL_TdNB9w
24    59dpWjJdntwzfIsKOcbQqA
25    xQBiULOOxAqn_y39Ynckbw
26    QqMQchSLRdodtUlrICXTjw
27    mijVMekO1vG7MbAL78uRNg
28    Guhi3rYZBLaXCwSVNKwSFw
29    uYSU2D0byI__7E1nLFx49w
30    I1OuDgYZOknPxp8MURUT1w
31    0D5WmoI0xv-t8PTDwhluhg
32    t1MblByl5GkJXIAIPzewQA
33    yuWOLUWnGu2XFiL_TdNB9w
34    59dpWjJd

In [95]:
review_df['user_id'] = review_df['user_id'].astype(str)
print(review_df['user_id'])

0     I1OuDgYZOknPxp8MURUT1w
1     0D5WmoI0xv-t8PTDwhluhg
2     t1MblByl5GkJXIAIPzewQA
3     yuWOLUWnGu2XFiL_TdNB9w
4     59dpWjJdntwzfIsKOcbQqA
5     xQBiULOOxAqn_y39Ynckbw
6     QqMQchSLRdodtUlrICXTjw
7     mijVMekO1vG7MbAL78uRNg
8     Guhi3rYZBLaXCwSVNKwSFw
9     uYSU2D0byI__7E1nLFx49w
10    I1OuDgYZOknPxp8MURUT1w
11    0D5WmoI0xv-t8PTDwhluhg
12    t1MblByl5GkJXIAIPzewQA
13    ul6pwQqpzj-pbmzMghDl7g
14    59dpWjJdntwzfIsKOcbQqA
15    xQBiULOOxAqn_y39Ynckbw
16    QqMQchSLRdodtUlrICXTjw
17    mijVMekO1vG7MbAL78uRNg
18    Guhi3rYZBLaXCwSVNKwSFw
19    uYSU2D0byI__7E1nLFx49w
20    I1OuDgYZOknPxp8MURUT1w
21    0D5WmoI0xv-t8PTDwhluhg
22    t1MblByl5GkJXIAIPzewQA
23    yuWOLUWnGu2XFiL_TdNB9w
24    59dpWjJdntwzfIsKOcbQqA
25    xQBiULOOxAqn_y39Ynckbw
26    QqMQchSLRdodtUlrICXTjw
27    mijVMekO1vG7MbAL78uRNg
28    Guhi3rYZBLaXCwSVNKwSFw
29    uYSU2D0byI__7E1nLFx49w
30    I1OuDgYZOknPxp8MURUT1w
31    0D5WmoI0xv-t8PTDwhluhg
32    t1MblByl5GkJXIAIPzewQA
33    yuWOLUWnGu2XFiL_TdNB9w
34    59dpWjJd

In [56]:
review_df['business_id'] = business_id_df['business_id'][2]


/var/folders/pn/q7mmpvpx5n36lk_j6p9gqk4c0000gn/T/ipykernel_3373/1828538604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['business_id'] = business_id_df['business_id'][2]


In [57]:
review_df = review_df[['business_id','user_id','name', 'stars', 'useful','funny','cool','text', 'place', 'date']]

In [58]:
review_df

,business_id,user_id,name,stars,useful,funny,cool,text,place,date
0,tUFrWirKiKi_TAnsVWINQQ,7ZbChk7xNMQ4WJh1vIhd5w,James S.,5.0,2,0,0,I never thought I'd spend time on Yelp reviewi...,Location,2021-08-03 12:19:49
1,tUFrWirKiKi_TAnsVWINQQ,Ce2zQ1OZXSLuB-9eqqUzww,Jessica G.,1.0,0,0,0,If I could put 0 stars I most definitely would...,"Tucson, AZ",2023-09-26 10:25:32
2,tUFrWirKiKi_TAnsVWINQQ,k-7UjkZ_jvi8HkhHbGrTuw,Trevor 2.,5.0,0,1,0,Every time I mention going here to my fiancé s...,"Tucson, AZ",2022-07-22 11:42:27
3,tUFrWirKiKi_TAnsVWINQQ,dhSgDUsVSBDhDeVtseljWw,Lindsey J.,2.0,0,0,0,I just tried calling 3 times pressing differen...,"Tucson, AZ",2023-07-16 00:47:50
4,tUFrWirKiKi_TAnsVWINQQ,3e7nJoExe3ddC5fuo0OihA,Sunny S.,1.0,0,1,0,"I was in the area and needed to kill time, so ...","Sabino Hills, Tucson, AZ",2022-09-23 17:07:43
...,...,...,...,...,...,...,...,...,...,...
65,tUFrWirKiKi_TAnsVWINQQ,2vkp7qElBzc0y1oiLBC9lQ,K. A.,1.0,0,0,0,Just went to find an anniversary card for my h...,"Tucson, AZ",2023-05-16 04:41:26
66,tUFrWirKiKi_TAnsVWINQQ,nZuH05cjCs7o1fY3OM1yNw,Jonnita W.,5.0,5,1,4,"This Arizona Target location, for me was my BE...","Tucson, AZ",2021-11-17 04:54:56
67,tUFrWirKiKi_TAnsVWINQQ,p_2UE4C4Xwx92YCuH2I19w,Andrés P.,1.0,1,0,0,"Honestly, I've held off from leaving a review ...","Stockton, CA",2021-12-03 08:35:07
68,tUFrWirKiKi_TAnsVWINQQ,XYDJ-NUaJjlu_fmxZV5vLw,Sean D.,2.0,0,1,0,What is going on? This Target has really gone ...,"Tucson, AZ",2022-04-08 21:02:33
